In [ ]:
# Software for Visualization of Patient Risk–Benefit Space
# ============================================

# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load Data

# >>> Update the file path to your Excel file
file_path = "book1.xlsx"

# Define sheet names for therapy groups (adjust if needed)
therapy_sheets = {
    'Mono': 'Benefit mono',      # Example sheet name for monotherapy
    'Combo': 'Benefit combo'     # Example sheet name for combination therapy
}

dfs = []
for therapy, sheet in therapy_sheets.items():
    df = pd.read_excel(file_path, sheet_name=sheet)
    df['Terapija'] = therapy
    dfs.append(df)

# Combine all sheets into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Determine the x-axis column automatically (first column)
x_column = combined_df.columns[0]  # Could be 'Benefits' or 'Adverse Events' or 'Risks'

In [ ]:
# Forest Plot Function


def plot_forest(data, ax, x_col):
    """
    Creates a forest plot where:
    - x-axis lists benefits or adverse events
    - y-axis shows point estimates with 95% confidence intervals
    Colors denote therapy groups.
    """
    ax.set_title("Forest plot: Risk–Benefit Visualization of Therapies", fontsize=22, pad=20)
    ax.set_ylabel("Point Estimate (95% CI)", fontsize=18)
    ax.set_xlabel(x_col, fontsize=18)  # Flexible x-axis label, depending on your analysis

    ax.set_axisbelow(True)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, zorder=0)

    x_labels = data[x_col].unique()
    text_offset = 0.05

    xticks = []
    xticklabels = []

    for i, label in enumerate(x_labels):
        xticks.append(i)
        xticklabels.append(label)
        label_data = data[data[x_col] == label]

        for _, row in label_data.iterrows():
            point_estimate = row['point estimate']
            ci_low = row['95% CI-left']
            ci_high = row['95% CI-right']
            therapy = row['Terapija']
            color = 'red' if therapy == 'Mono' else 'blue'
            x_pos = i

            ax.plot([x_pos, x_pos], [ci_low, ci_high], color=color, lw=2, zorder=3)
            ax.scatter(x_pos, point_estimate, color=color, zorder=4)

            if therapy == 'Mono':
                ax.text(x_pos - text_offset, ci_low, f"{ci_low:.2f}", va='center', ha='right', fontsize=10, color='red')
                ax.text(x_pos - text_offset, ci_high, f"{ci_high:.2f}", va='center', ha='right', fontsize=10, color='red')
            else:
                ax.text(x_pos + text_offset, ci_low, f"{ci_low:.2f}", va='center', ha='left', fontsize=10, color='blue')
                ax.text(x_pos + text_offset, ci_high, f"{ci_high:.2f}", va='center', ha='left', fontsize=10, color='blue')

    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels, rotation=0, ha='center', fontsize=14)

    # Optional: remove overlapping y-ticks 
    # (here "SD" as an example, e.g., Stable Disease from benefits data) 
    # to improve readability; adjust or remove as needed for your dataset
    if "SD" in data[x_col].unique():
        sd_point = data[data[x_col] == 'SD']['point estimate'].iloc[0]
        all_point_estimates = data['point estimate'].unique()
        filtered_ys = sorted([pe for pe in all_point_estimates if not np.isclose(pe, sd_point)])
        ax.set_yticks(filtered_ys)
        ax.set_yticklabels([f"{pe:.2f}" for pe in filtered_ys], fontsize=12)

In [ ]:
# Generate Visualization

fig, ax = plt.subplots(figsize=(12, 10))
plot_forest(combined_df, ax, x_column)

# Add legend
mono_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Monotherapy')
combo_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Combination therapy')
ax.legend(handles=[mono_patch, combo_patch], loc='upper right', prop={'size': 14})

plt.tight_layout()
plt.show()

In [ ]:
# Save Results (Optional)

# plt.savefig("risk_benefit_forestplot.png", dpi=300)